# Laptop vs. Mobile Viewership

## NY Times SQL Interview Question

### Question

This is the same question as problem #3 in the SQL Chapter of Ace the Data Science Interview!

Assume you're given the table on user viewership categorized by device type where the three types are laptop, tablet, and phone.

Write a query that calculates the total viewership for laptops and mobile devices where mobile is defined as the sum of tablet and phone viewership. Output the total viewership for laptops as `laptop_reviews` and the total viewership for mobile devices as `mobile_views`.

---

### Effective 15 April 2023, the solution has been updated with a more concise and easy-to-understand approach.

---

### Table: `viewership`

| Column Name | Type     |
|-------------|----------|
| user_id     | integer  |
| device_type | string   |
| view_time   | timestamp|

---

### Example Input for `viewership` Table:

| user_id | device_type | view_time           |
|---------|-------------|---------------------|
| 123     | tablet      | 01/02/2022 00:00:00 |
| 125     | laptop      | 01/07/2022 00:00:00 |
| 128     | laptop      | 02/09/2022 00:00:00 |
| 129     | phone       | 02/09/2022 00:00:00 |
| 145     | tablet      | 02/24/2022 00:00:00 |

---

### Example Output:

| laptop_views | mobile_views |
|--------------|--------------|
| 2            | 3            |

---

### Explanation

Based on the example input, there are a total of **2 laptop views** and **3 mobile views** (the sum of tablet and phone views).


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from datetime import datetime

# Create Spark session
spark = SparkSession.builder.master('local[1]').getOrCreate()
sc = spark.sparkContext

# Define the data for viewership table
df = sc.parallelize([
    (123, "tablet", datetime(2022, 1, 2, 0, 0)),
    (125, "laptop", datetime(2022, 1, 7, 0, 0)),
    (128, "laptop", datetime(2022, 2, 9, 0, 0)),
    (129, "phone", datetime(2022, 2, 9, 0, 0)),
    (145, "tablet", datetime(2022, 2, 24, 0, 0))
])



# Show the DataFrame
df.toDF().show(truncate=False)


+---+------+-------------------+
|_1 |_2    |_3                 |
+---+------+-------------------+
|123|tablet|2022-01-02 00:00:00|
|125|laptop|2022-01-07 00:00:00|
|128|laptop|2022-02-09 00:00:00|
|129|phone |2022-02-09 00:00:00|
|145|tablet|2022-02-24 00:00:00|
+---+------+-------------------+



In [3]:
def classify_device(row):  
    if row[1] in ['phone', 'tablet']:
        return ('mobile_views', 1)
    elif row[1] == 'laptop':
        return ('laptop_views', 1)
    else:
        return ('other', 0)  # optional

df2 = df.map(classify_device)\
        .reduceByKey(lambda a, b: a + b)
            

df2.toDF().show()


+------------+---+
|          _1| _2|
+------------+---+
|mobile_views|  3|
|laptop_views|  2|
+------------+---+

